### IMPORTS

In [1]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser
from langchain.chains import LLMChain, SequentialChain
from langchain_pinecone import PineconeVectorStore 
from langchain_community.document_loaders import DirectoryLoader 
from langchain_text_splitters import RecursiveCharacterTextSplitter 
from langchain_experimental.sql import SQLDatabaseChain
from langchain.sql_database import SQLDatabase
import os 
import glob
from pathlib import Path
from dotenv import load_dotenv
import os

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
# deepseek_api_key = os.getenv("DEEPSEEK_API_KEY")
pinecone_api_key = os.getenv("PINECONE_API_KEY")
# print(deepseek_api_key)


c:\Users\tirth\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### SELECT YOUR MODEL

In [2]:
# Initialize the OpenAI model via LangChain
llm = ChatOpenAI(
    model="gpt-4o-mini-2024-07-18",
    api_key=openai_api_key
)

# llm = ChatOpenAI(
#     model="deepseek-chat",
#     api_key=deepseek_api_key
# )

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"
)

index_name = "mark-3"

database = SQLDatabase.from_uri("sqlite:///database/tirth.db")





### Loading Documents

In [3]:
loader = DirectoryLoader('source',glob="**/*.pdf")
docs = loader.load()
docs[0]

Document(metadata={'source': 'source\\legion_pro_5i.pdf'}, page_content='User Guide\n\nLenovo Legion Pro 5i (16″, 8) and Lenovo Legion Pro 5 (16″, 8)\n\nRead this first\n\nBefore using this documentation and the product it supports, ensure that you read and understand the following:\n\nGeneric Safety and Compliance Notices\n\nSafety and Warranty Guide\n\nSetup Guide\n\nFirst Edition (February 2023)\n\n© Copyright Lenovo 2023.\n\nLIMITED AND RESTRICTED RIGHTS NOTICE: If data or software is delivered pursuant to a General Services Administration “GSA” contract, use, reproduction, or disclosure is subject to restrictions set forth in Contract No. GS- 35F-05925.\n\nContents\n\nAbout this guide . . . . . . . . . . . . . iii\n\nChapter 1. Meet your computer. . . . . 1 . . . . . . 1 Front . . . . . . . 2 Base . . . . . . . 3 Left . . . . . 4 . . Right . . 5 . . . Rear . . . . 6 . . . Bottom . . . 7 Features and specifications . . . 8 . 8\n\n. . . . . . . . . . . . . . . . . . . . . . Statemen

### Chunking Documents

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1400,chunk_overlap=200)
split_docs = text_splitter.split_documents(docs)
len(split_docs[1].page_content)
len(split_docs)
vectorstore = PineconeVectorStore.from_documents(split_docs, embeddings, index_name=index_name)

48

### Initialize retriver

In [9]:
vectorstore = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings,
)

## HELPER FUNCTIONS
#### RETRIVER 

In [ ]:
def get_context(query):
    similar_docs = vectorstore.similarity_search(query, k=3)
    context = [' '.join(doc.page_content.split()) for doc in similar_docs]
    return context

# get_context("input socket")

### Query for Retriving Context

In [11]:
user_input = """
How many connection ports does the laptop have ?
"""

retrival_template = f"""Rewrite the following query so that it can be used as an input prompt in a retriver of a Vector DB. The new query should:
- DO NOT ANSWER the original query; only rewrite it.

Return ONLY the rewritten query text without additional formatting or explanations.

Original query: 

"""

# Create ChatPromptTemplate for the rewriting chain
retrival_prompt = ChatPromptTemplate.from_messages([
    ("system", retrival_template),
    ("human", "{text}")
])

# Now wrap each prompt into an LLMChain
retrival_chain = LLMChain(
    llm=llm,
    prompt=retrival_prompt,
    output_key="retrival_query"
)

result = retrival_chain.invoke({"text":user_input})
print( "retrival_prompt_Question : ", result["retrival_query"])

retrived_context = get_context(result["retrival_query"])
print("retrived Context : ")
[print(x) for x in retrived_context]


C:\Users\tirth\AppData\Local\Temp\ipykernel_24700\458303865.py:21: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  retrival_chain = LLMChain(


retrival_prompt_Question :  What is the number of connection ports available on the laptop?
retrived Context : 
– Thickest: 26.75 mm (1.1 inches) ac power adapter Input: 100 V ac–240 V ac, 50 Hz–60 Hz Output: 20 V dc, 11.5 A or 15 A Power: 230 W or 300 W Battery pack Capacity: 80 Wh Number of cells: 4 Note: The battery capacity is the typical or average capacity as measured in a specific test environment. Capacities measured in other environments may differ but are no lower than the rated capacity (see product label). Microprocessor To view the microprocessor information of your computer: Right-click the Start button and then select System. Type system information in the Windows search box and then press Enter. Memory Type: Double data rate 5 (DDR5), small outline dual in-line memory module (SODIMM) Number of physical slots: 2 Secondary storage device Type: solid-state drive Form factor: M.2 (2242 or 2280) Number of slots: 2 Bus: PCI Express Screen Size: 406.4 mm (16 inches) Display re

[None, None, None]

### Sqlite3


In [38]:
user_query = """Give me the avg money spent by a customer along with more details """
user_query = user_query.strip() + " In tabular format"
print(user_query)
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=database,llm=llm)

agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_executor_kwargs={
        "handle_parsing_errors": True
    },
)

final = agent.run(user_query)
print(type(final))
print(final)

Give me the avg money spent by a customer along with more details In tabular format


> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables  
Action Input: ""  categories, customers, employees, order_items, orders, payments, product_categories, products, shippingIt looks like the relevant tables for customer spending will be `customers`, `orders`, and `payments`. I will check the schema for these tables to understand the columns and then formulate a query for the average money spent by a customer including additional details.

Action: sql_db_schema  
Action Input: customers, orders, payments  
CREATE TABLE customers (
	customer_id INTEGER, 
	first_name TEXT NOT NULL, 
	last_name TEXT NOT NULL, 
	email TEXT NOT NULL, 
	phone TEXT, 
	address TEXT, 
	PRIMARY KEY (customer_id), 
	UNIQUE (email)
)

/*
3 rows from customers table:
customer_id	first_name	last_name	email	phone	address
1	John	Doe	john.doe@example.com	555-1234	123 Maple Street
2	Jane	Smith	jane.smith@example.com	

### Rewrite Query for Final Question

In [12]:

rewrite_template = f"""Rewrite the following query so that it can be used as an input prompt in a RAG system. The new query should:
- Preserve the core intent and meaning of the original query.
- Try to Question the same Questions in different manner 
- Break down the query into steps that lead to an answer.
- Expand and clarify the query to be more specific and informative for retrieving relevant context.
- Avoid introducing new topics or deviating from the original query.
- DO NOT ANSWER the original query; only rewrite it.

Return ONLY the rewritten query text without additional formatting or explanations.

Original query: 

"""

# Create ChatPromptTemplate for the rewriting chain
rewrite_prompt = ChatPromptTemplate.from_messages([
    ("system", rewrite_template),
    ("human", "{text}")
])

# Now wrap each prompt into an LLMChain
rewrite_chain = LLMChain(
    llm=llm,
    prompt=rewrite_prompt,
    output_key="rewritten_query"
)

result = rewrite_chain.invoke({"text":user_input})
final_prompt_question = result["rewritten_query"]
print("final_prompt_question : ",final_prompt_question)

final_prompt_question :  What is the total number of connection ports available on the laptop? Could you specify what types of ports are included in this count? Additionally, are there any unique features or specifications related to these connection ports that I should know about?


In [13]:
system_prompt = f"""You are a helpful assistant that is an expert at extracting the most useful information from a given text.
Also bring in extra relevant information to the user query from outside the given context only if the given information is not enough.
And if you solve the question by using the extra relevant information that you brought, please do wrap it in <suggested> </suggested> tag.

Context: {retrived_context}
"""


# Create ChatPromptTemplate for the main answering chain
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{text}")
])


answer_chain = LLMChain(
    llm=llm,
    prompt=chat_prompt,
    output_key="answer"
)

final_result = answer_chain.invoke({"text":final_prompt_question})
print(final_result["answer"])


The total number of connection ports available on the laptop can be calculated based on the information provided:

1. **USB Type-A Connectors**: 4 ports
2. **Multi-purpose USB Type-C Connectors**: 2 ports
3. **Combo Audio Jack**: 1 port
4. **HDMI Connector**: 1 port
5. **RJ45 Connector (Ethernet)**: 1 port

**Total number of connection ports = 4 (USB Type-A) + 2 (USB Type-C) + 1 (Audio Jack) + 1 (HDMI) + 1 (RJ45) = 9 ports**

### Types of Ports:
- **USB Type-A**: 4 ports, one of which has an Always-On function for charging.
- **Multi-purpose USB Type-C**: 2 ports supporting SuperSpeed USB 10 Gbps and DisplayPort Alt Mode, with power output capabilities.
- **Combo Audio Jack**: 1 port, used for headphones or headset and does not support standalone external microphones.
- **HDMI Connector**: 1 port, supports resolutions up to 7680 × 4320 at 60 Hz.
- **RJ45 Connector**: 1 port for Ethernet networking.

### Unique Features:
- **USB Type-A Ports**: One Type-A port has an Always-On feature t

# quick fix

### history of chat 
### get reading from the reviews.
### suggests improvements from the reviews 



### turn it into a AGENT